<table style="width:100%">
  <tr>
    <td><center style="font-size:500%;">Representação Distribuida de Palavras (Parte 1)</center></td>
    <td><img src="https://logodownload.org/wp-content/uploads/2015/02/puc-rio-logo.gif" width="100"/></td> 
  </tr>    
</table>



In [ ]:
import numpy as np
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity

#carrega um modelo pré-treinado (gloVe word vectors)
word_vectors = api.load('glove-wiki-gigaword-50')

# função de word -> vec
word2vec = lambda w: word_vectors.word_vec(w.lower())

#função de similaridade
similarity = lambda x,y:cosine_similarity(x[np.newaxis,:],y[np.newaxis,:]).squeeze()

[==================================================] 100.0% 66.0/66.0MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
word_a, word_b = "father", "mother"
word_a, word_b = "ball", "crocodile"

vec_a = word2vec(word_a) #transforma em word2vec
vec_b = word2vec(word_b) #transforma em word2vec

#calcula similaridade entre vec_a e vec_b
result = similarity(vec_a, vec_b).squeeze()
print("cosine_similarity( {} , {} ): {:.4f}".format(word_a,word_b,result))

cosine_similarity( ball , crocodile ): 0.2744


In [ ]:
pos_a, neg_a = "france" , "paris"
pos_a_v = word2vec(pos_a)
neg_a_v = word2vec(neg_a)

pos_b, neg_b = "italy", "rome"
pos_b_v = word2vec(pos_b)
neg_b_v = word2vec(neg_b)

#Quão similar são as palavras:

result = similarity(pos_a_v - neg_a_v, pos_b_v - neg_b_v)
print("cosine_similarity( {} - {} , {} - {} ): {:.4f}".format(pos_a, neg_a, pos_b, neg_b, result))

result = similarity(pos_a_v - neg_a_v, neg_b_v - pos_b_v)
print("cosine_similarity( {} - {} , {} - {} ): {:.4f}".format(pos_a, neg_a, neg_b, pos_b, result))

cosine_similarity( france - paris , italy - rome ): 0.6751
cosine_similarity( france - paris , rome - italy ): -0.6751


In [ ]:
def complete_analogy(pos_a, neg_a, pos_b):
    
    pos_a_v, neg_a_v, pos_b_v = word2vec(pos_a), word2vec(neg_a), word2vec(pos_b)
    d = pos_a_v - neg_a_v
    
    max_cosine_sim = -100              
    best_word = None                   

    for neg_b in word_vectors.vocab:        
        
        if neg_b in [pos_a, neg_a, pos_b] :
            continue
            
        neg_b_v = word2vec(neg_b)
        cosine_sim = similarity(d , pos_b_v - neg_b_v)
        
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = neg_b
            
    return best_word

In [ ]:
analogies_to_try = [('italy', 'italian', 'ARGENTINA'), 
                 ('india', 'delhi', 'japan'), 
                 ('man', 'woman', 'boy'), 
                 ('small', 'smaller', 'large')]

# dadas 3 palavras, encontre a próxima palavra
for analogy in analogies_to_try:
    print ('{} -> {} :: {} -> {}'.format( *analogy, complete_analogy(*analogy)))

italy -> italian :: ARGENTINA -> argentine
india -> delhi :: japan -> tokyo
man -> woman :: boy -> girl


In [ ]:
g = word2vec('woman') - word2vec('man')
print(g)

In [ ]:
#Nomes masculinos são negativos
print ('Lista de nomes e sua similaridade com o vetor g:')

name_list = ['john', 'marie', 'sophie', 'ronaldo', 'priya', 'rahul', 'danielle', 'reza', 'katy', 'yasmin']

for word in name_list:
    print("{}:{}".format(word, similarity(word2vec(word), g)))

In [ ]:
print('Outras palavras e suas similaridades:')
word_list = ['lipstick', 'guns', 'science', 'arts', 'literature', 'warrior','doctor', 'tree', 'receptionist', 
             'technology',  'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']
for word in word_list:
    print("{}:{}".format(word, similarity(word2vec(word), g)))